**Set environment**

In [1]:
### set env
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY:     /data/reddylab/Kuei
WORK DIRECTORY:     /data/reddylab/Kuei/out
CODE DIRECTORY:     /data/reddylab/Kuei/code
SING DIRECTORY:     /data/reddylab/Kuei/singularity
PATH OF SOURCE:     /data/reddylab/Kuei/source
PATH OF EXECUTABLE: /data/reddylab/Kuei/bin
PATH OF ANNOTATION: /data/reddylab/Kuei/annotation
PATH OF PROJECT:    /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS:    /data/reddylab/Kuei/out/proj_combeffect_encode_fcc



**Check files**

In [2]:
ls ${FD_RES}/region

annotation_cCREs           annotation_genehancer  KS91_K562_ASTARRseq_peak_macs
annotation_enhancer_junke  annotation_motif
annotation_gencode         annotation_remap2022


In [3]:
ls ${FD_RES}/region/KS91_K562_ASTARRseq_peak_macs

KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.ccre_all.bed.gz
KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.enhancer_junke_peak_astarr.bed.gz
KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.enhancer_junke_peak_wstarr.bed.gz
KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.gencode_v24_tss.bed.gz
KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.nuc.bed.gz
KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.remap2022_nr.bed.gz
KS91_K562_hg38_ASTARRseq_Input.ccre_all.freq.tsv
KS91_K562_hg38_ASTARRseq_Input.count.ccre_all.tsv
KS91_K562_hg38_ASTARRseq_Input.count.enhancer_junke_peak_astarr.tsv
KS91_K562_hg38_ASTARRseq_Input.count.enhancer_junke_peak_wstarr.tsv
KS91_K562_hg38_ASTARRseq_Input.count.gencode_v24_tss.tsv
KS91_K562_hg38_ASTARRseq_Input.count.remap2022_nr.tsv
KS91_K562_hg38_ASTARRseq_Input.peaks.MYC.FA

In [4]:
ls ${FD_RES}/region/annotation_genehancer

hub_3160057_GeneHancer_double_elite_hg38.bed
hub_3160057_GeneHancer_double_elite_hg38_chr11.bed
hub_3160057_GeneHancer_double_elite_hg38_chr11.tsv
hub_3160057_GeneHancer_double_elite_hg38_chr8.bed
hub_3160057_GeneHancer_double_elite_hg38_chr8.tsv
hub_3160057_GeneHancer_double_elite_hg38_chrX.bed
hub_3160057_GeneHancer_double_elite_hg38_chrX.tsv
hub_3160057_GeneHancer_hg38.bed
hub_3160057_GeneHancer_hg38_chr11.bed
hub_3160057_GeneHancer_hg38_chr11.tsv
hub_3160057_GeneHancer_hg38_chr8.bed
hub_3160057_GeneHancer_hg38_chr8.tsv
hub_3160057_GeneHancer_hg38_chrX.bed
hub_3160057_GeneHancer_hg38_chrX.tsv


## Annotation with GeneHancer

In [28]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 20G \
    --output ${FD_LOG}/annotation_astarr_peak_macs_input_hub_3160057_GeneHancer.txt \
    --array 0 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh

### set input and output
FD_BED=${FD_RES}/region/KS91_K562_ASTARRseq_peak_macs
FN_BED=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
FP_BED=${FD_BED}/${FN_BED}

FD_ANN=${FD_RES}/region/annotation_genehancer
FN_ANN=hub_3160057_GeneHancer_hg38.bed
FP_ANN=${FD_ANN}/${FN_ANN}

FD_OUT=${FD_RES}/region/KS91_K562_ASTARRseq_peak_macs
FN_OUT=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.${FN_ANN}.gz
FP_OUT=${FD_OUT}/${FN_OUT}

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### show I/O file
echo "Input: " ${FP_BED}
echo
echo "show first few lines of input"
cat ${FP_BED} | head -5
echo
echo "Input: " ${FP_ANN}
echo
echo "show first few lines of input"
cat ${FP_ANN} | head -5
echo

### init: create output folder if not exist
mkdir -p ${FD_OUT}

### annotation using intersect
bedtools intersect \
    -a ${FP_BED} \
    -b ${FP_ANN} \
    -wo \
    -f 0.5 \
    -F 0.5 \
    -e \
| gzip -c \
> ${FP_OUT}

### show output file
echo
echo "Output: " ${FP_OUT}
echo
echo "show first few lines of output:"
zcat ${FP_OUT} | head -5
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 29444815


In [29]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 20G \
    --output ${FD_LOG}/annotation_astarr_peak_macs_input_hub_3160057_GeneHancer_double_elite.txt \
    --array 0 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh

### set input and output
FD_BED=${FD_RES}/region/KS91_K562_ASTARRseq_peak_macs
FN_BED=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
FP_BED=${FD_BED}/${FN_BED}

FD_ANN=${FD_RES}/region/annotation_genehancer
FN_ANN=hub_3160057_GeneHancer_double_elite_hg38.bed
FP_ANN=${FD_ANN}/${FN_ANN}

FD_OUT=${FD_RES}/region/KS91_K562_ASTARRseq_peak_macs
FN_OUT=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.${FN_ANN}.gz
FP_OUT=${FD_OUT}/${FN_OUT}

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### show I/O file
echo "Input: " ${FP_BED}
echo
echo "show first few lines of input"
cat ${FP_BED} | head -5
echo
echo "Input: " ${FP_ANN}
echo
echo "show first few lines of input"
cat ${FP_ANN} | head -5
echo

### init: create output folder if not exist
mkdir -p ${FD_OUT}

### annotation using intersect
bedtools intersect \
    -a ${FP_BED} \
    -b ${FP_ANN} \
    -wo \
    -f 0.5 \
    -F 0.5 \
    -e \
| gzip -c \
> ${FP_OUT}

### show output file
echo
echo "Output: " ${FP_OUT}
echo
echo "show first few lines of output:"
zcat ${FP_OUT} | head -5
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 29444816


In [30]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 20G \
    --output ${FD_LOG}/annotation_astarr_peak_macs_input_geneHancerRegElements.txt \
    --array 0 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh

### set input and output
FD_BED=${FD_RES}/region/KS91_K562_ASTARRseq_peak_macs
FN_BED=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
FP_BED=${FD_BED}/${FN_BED}

FD_ANN=${FD_RES}/region/annotation_genehancer
FN_ANN=geneHancerRegElements_hg38.bed
FP_ANN=${FD_ANN}/${FN_ANN}

FD_OUT=${FD_RES}/region/KS91_K562_ASTARRseq_peak_macs
FN_OUT=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.${FN_ANN}.gz
FP_OUT=${FD_OUT}/${FN_OUT}

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### show I/O file
echo "Input: " ${FP_BED}
echo
echo "show first few lines of input"
cat ${FP_BED} | head -5
echo
echo "Input: " ${FP_ANN}
echo
echo "show first few lines of input"
cat ${FP_ANN} | head -5
echo

### init: create output folder if not exist
mkdir -p ${FD_OUT}

### annotation using intersect
bedtools intersect \
    -a ${FP_BED} \
    -b ${FP_ANN} \
    -wo \
    -f 0.5 \
    -F 0.5 \
    -e \
| gzip -c \
> ${FP_OUT}

### show output file
echo
echo "Output: " ${FP_OUT}
echo
echo "show first few lines of output:"
zcat ${FP_OUT} | head -5
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 29444817


In [31]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 20G \
    --output ${FD_LOG}/annotation_astarr_peak_macs_input_geneHancerRegElementsDoubleElite.txt \
    --array 0 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh

### set input and output
FD_BED=${FD_RES}/region/KS91_K562_ASTARRseq_peak_macs
FN_BED=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
FP_BED=${FD_BED}/${FN_BED}

FD_ANN=${FD_RES}/region/annotation_genehancer
FN_ANN=geneHancerRegElementsDoubleElite_hg38.bed
FP_ANN=${FD_ANN}/${FN_ANN}

FD_OUT=${FD_RES}/region/KS91_K562_ASTARRseq_peak_macs
FN_OUT=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.${FN_ANN}.gz
FP_OUT=${FD_OUT}/${FN_OUT}

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### show I/O file
echo "Input: " ${FP_BED}
echo
echo "show first few lines of input"
cat ${FP_BED} | head -5
echo
echo "Input: " ${FP_ANN}
echo
echo "show first few lines of input"
cat ${FP_ANN} | head -5
echo

### init: create output folder if not exist
mkdir -p ${FD_OUT}

### annotation using intersect
bedtools intersect \
    -a ${FP_BED} \
    -b ${FP_ANN} \
    -wo \
    -f 0.5 \
    -F 0.5 \
    -e \
| gzip -c \
> ${FP_OUT}

### show output file
echo
echo "Output: " ${FP_OUT}
echo
echo "show first few lines of output:"
zcat ${FP_OUT} | head -5
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 29444818


In [32]:
cat ${FD_LOG}/annotation_astarr_peak_macs_input_hub_3160057_GeneHancer.txt

Hostname:           x1-01-3.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         01-11-23+03:09:27

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_ASTARRseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

show first few lines of input
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/annotation_genehancer/hub_3160057_GeneHancer_hg38.bed

show first few lines of input
chr8	62000	62200	GH08J000063	1	.	62000	62200	192,192,192	Ensembl	Enhancer	-
chr8	62600	62800	GH08J000062	1	.	62600	62800	192,192,192	Ensembl	Enhancer	-
chr8	63800	64000	GH08J000065	1	.	63800	64000	192,192,192	Ensembl	Enhancer	-
chr8	65000	65400	GH08J000064	1	.	65000	65400	192,192,192	Ensembl	Enhancer	-
chr8	70000	70200	GH08J000069	1	.	70000	70200	192,192,192	Ensembl	Enhancer	-


Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_A

In [33]:
cat ${FD_LOG}/annotation_astarr_peak_macs_input_hub_3160057_GeneHancer_double_elite.txt

Hostname:           x1-01-3.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         01-11-23+03:09:57

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_ASTARRseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

show first few lines of input
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/annotation_genehancer/hub_3160057_GeneHancer_double_elite_hg38.bed

show first few lines of input
chr8	230200	234401	GH08J000231	500	.	230200	234401	204,0,0	EPDnew,Ensembl,ENCODE(Z-Lab),CraniofacialAtlas	Promoter/Enhancer	Elite
chr8	272202	275574	GH08J000272	278	.	272202	275574	95,95,95	ENCODE(Z-Lab),CraniofacialAtlas	Enhancer	Elite
chr8	321600	322000	GH08J000320	48	.	321600	322000	160,160,160	Ensembl,dbSUPER	Enhancer	Elite
chr8	329200	330200	GH08J000331	105	.	329200	330200	160,160,160	Ensembl,dbSUPER	Enhancer	Elite
chr8	354200	354994	GH08

In [34]:
cat ${FD_LOG}/annotation_astarr_peak_macs_input_geneHancerRegElements.txt

Hostname:           x1-01-4.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         01-11-23+03:09:57

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_ASTARRseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

show first few lines of input
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/annotation_genehancer/geneHancerRegElements_hg38.bed

show first few lines of input
chr8	90065	91199	GH08J000090	1	.	90065	91199	192,192,192	Ensembl	Enhancer	-
chr8	120400	121400	GH08J000120	1	.	120400	121400	192,192,192	Ensembl	Enhancer	-
chr8	204199	204601	GH08J000204	1	.	204199	204601	192,192,192	Ensembl	Enhancer	-
chr8	211999	212400	GH08J000212	105	.	211999	212400	160,160,160	Ensembl	Enhancer	-
chr8	214400	215000	GH08J000214	36	.	214400	215000	192,192,192	Ensembl	Enhancer	-


Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/

In [35]:
cat ${FD_LOG}/annotation_astarr_peak_macs_input_geneHancerRegElementsDoubleElite.txt

Hostname:           x1-02-1.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         01-11-23+03:09:57

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_ASTARRseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

show first few lines of input
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/annotation_genehancer/geneHancerRegElementsDoubleElite_hg38.bed

show first few lines of input
chr8	230999	233601	GH08J000231	553	.	230999	233601	255,51,51	ENCODE(Z-Lab),EPDnew,Ensembl	Promoter/Enhancer	Elite
chr8	376140	378876	GH08J000376	125	.	376140	378876	160,160,160	ENCODE(Z-Lab),Ensembl,dbSUPER	Enhancer	Elite
chr8	399878	402065	GH08J000399	241	.	399878	402065	95,95,95	ENCODE(Z-Lab),dbSUPER	Enhancer	Elite
chr8	460815	463331	GH08J000460	133	.	460815	463331	160,160,160	ENCODE(Z-Lab),Ensembl	Enhancer	Elite
chr8	486330	487638	GH08J000486	